# Push
This notebook simulates the Object Detection and Tracking component. It sends messages about two objects - the first one moves in a square, whereas the second one moves in a circle.

In [ ]:
import socket
import time
import json
import numpy as np
import math
import sys

from time import time, sleep
from random import randint, random


In [ ]:
# Connection parameters
HOST = '127.0.0.1'   # The server's hostname or IP address
PORT = 65432         # The port used by the server
DATA_BUFF = 4096
SHUTDOWN_AFTER = 5000 # How many steps shall be broadcasted


# Object parameters

# Object types:
#    0 - Unknown
#    1 - Person
#    2 - Bicycle
#    3 - Car
#    4 - Motorcycle
#    5 - Bus
#    6 - Truck
RECT_OBJ_TYPE = 3
CIRCLE_OBJ_TYPE = 1 


# Coordinates for an object moving in a rectangle
LAT_MIN = 38.979056
LAT_MAX = 38.979965

LON_MIN = -76.925973
LON_MAX = -76.924568

# Coordinates for an object moving in a circle
LAT_CENTER = 38.98
LON_CENTER = -76.925
RADIUS = 0.0003


SPEED = 5.0 # Speed of objects (m/s)
ELEV = 25.0 # Elevation of the intersection (m.a.s.l)

NR_STEPS = 20 # How many steps (messages) to drive around the entire rectangle or circle
allsteps = 1

In [ ]:

def geo2Angle(lat1, lon1, lat2, lon2):
    dLon = (lon2 - lon1)

    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)

    brng = math.atan2(y, x)

    brng = math.degrees(brng)
    brng = (brng + 360) % 360
    #brng = 360 - brng # count degrees clockwise - remove to make counter-clockwise
    return brng

def circle(lat_center = LAT_CENTER, 
           lon_center = LON_CENTER, 
           radius = RADIUS,
           nr_steps = NR_STEPS):
    lat_min = lat_center - radius
    lat_max = lat_center + radius
    lon_min = lon_center - radius
    lon_max = lon_center + radius
    
    
    lat = lat_min
    step = (lat_max - lat_min) / nr_steps / 2
    
    while True:
        while lat < lat_max:
            lat += step
            lon = lon_center + math.sqrt(max(0, radius**2 - (lat-lat_center) ** 2))
            #print (radius, lat, lat_center,  lat-lat_center)
            lat = np.round(lat, 6)
            lon = np.round(lon, 6)
            yield lat, lon
        while lat > lat_min:
            lat -= step
            lon = lon_center - math.sqrt(max(0, radius**2 - (lat-lat_center) ** 2))
            lat = np.round(lat, 6)
            lon = np.round(lon, 6)
            yield lat, lon
    


def rect(lon_min = LON_MIN, 
         lon_max = LON_MAX, 
         lat_min = LAT_MIN, 
         lat_max = LAT_MAX, 
         nr_steps = NR_STEPS
        ):
    lat = lat_min
    lon = lon_min
    lat_step = (lat_max - lat_min) / nr_steps / 4
    lon_step = (lon_max - lon_min) / nr_steps / 4
    
    
    
    while True:
        while lat < lat_max:
            lat += lat_step
            lat = np.round(lat, 6)
            yield lat, lon
        while lon < lon_max:
            lon += lon_step
            lon = np.round(lon, 6)
            yield lat, lon
        while lat > lat_min:
            lat -= lat_step
            lat = np.round(lat, 6)
            yield lat, lon
        while lon > lon_min:
            lon -= lon_step
            lon = np.round(lon, 6)
            yield lat, lon



In [ ]:
def sendLatLon(s, lat, lon, obj_id, objType = 0, prevPoint = None):
    if prevPoint is None:
        heading = 0
    else:
        heading = geo2Angle(prevPoint[0], prevPoint[1],
                            lat, lon);

    data = {
        'id' : obj_id,
        'objectType' : objType,
        'secMark' : time(),
        'lat' : lat,
        'lon' : lon,
        'heading' : heading,
    }    
    data = {
        'mode' : 'push',
        'msg' : json.dumps(data)
    }
    msg = json.dumps(data)
    msg = str.encode(msg)
    s.sendall(msg)
    sys.stdout.write("Packet: {}, Sent: {}        \r".format(counter, msg))
    data = s.recv(1024)  
    return data

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    counter = 0
    prevPointRect = None
    prevPointCircle = None
    my_rectangle = rect()
    my_rectangle = circle(LAT_CENTER - 3*RADIUS, LON_CENTER - 3*RADIUS)
    my_circle = circle()
    while True:

        if random() < 0.9:
            lat, lon = next(my_rectangle)
            data = sendLatLon(s, lat, lon, 1, objType = RECT_OBJ_TYPE, prevPoint = prevPointRect)
            prevPointRect = (lat, lon)

        if random() < 0.9:
            lat, lon = next(my_circle)
            data = sendLatLon(s, lat, lon, 2, objType = CIRCLE_OBJ_TYPE, prevPoint = prevPointCircle)
            
        counter += 1
        if counter > SHUTDOWN_AFTER:
            s.close()
            break
            
        sleep(0.1)    

In [ ]:
s.close()